In [2]:
import requests
import pandas as pd

def fetch_hdi_data(api_key: str, start_year: int = 2022, n_years: int = 3) -> pd.DataFrame:
    base_url = "https://hdrdata.org/api"
    years = ','.join(str(year) for year in range(start_year, start_year - n_years, -1))
    
    try:
        countries_response = requests.get(f"{base_url}/Metadata/Countries", params={'apikey': api_key})
        countries_response.raise_for_status()
        country_codes = [c['code'] for c in countries_response.json() if c.get('code') and len(c['code']) == 3]
        
        data_response = requests.get(
            f"{base_url}/CompositeIndices/query",
            params={
                'apikey': api_key,
                'countryOrAggregation': ','.join(country_codes),
                'year': years,
                'indicator': 'HDI'
            }
        )
        data_response.raise_for_status()
        
        results = []
        for record in data_response.json():
            try:
                country_parts = record['country'].split(' - ', 1)
                iso3 = country_parts[0] if len(country_parts) == 2 else None
                country_name = country_parts[1] if len(country_parts) == 2 else country_parts[0]
                
                if iso3 and record.get('value') and record.get('year'):
                    results.append({
                        'country': country_name,
                        'iso_3': iso3,
                        'year': int(record['year']),
                        'value': float(record['value'])
                    })
            except (KeyError, ValueError, TypeError):
                continue
        
        df = pd.DataFrame(results)
        return df.sort_values(['country', 'year'], ascending=[True, False]).reset_index(drop=True)
    
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame(columns=['country', 'iso_3', 'year', 'value'])

if __name__ == "__main__":
    api_key = "YOUR_API_KEY"
    df = fetch_hdi_data(api_key)
    
    if not df.empty:
        df.to_csv('df_HDI.csv', index=False)
        print(f"Saved {len(df)} records to df_HDI.csv")

Saved 577 records to df_HDI.csv
